In [1]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
new_states = {v:k for k,v in us_state_abbrev.items()}

In [2]:
import os
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

In [3]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [4]:
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [5]:
#import zipfile
#!unzip/content/US_Accidents_Dec20_Updated.zip

In [6]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [7]:
accidents = pd.read_csv('US_Accidents_Dec20_Updated.csv')
dropthese = ['ID', 'End_Lat', 'End_Lng', 'End_Time', 'Description', 'Distance(mi)', 'Number', 'Street', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp']
accidents = accidents.drop(columns = dropthese, axis = 1)
accidents['Start_Time'] = pd.to_datetime(accidents['Start_Time'])
accidents.head()

,Severity,Start_Time,Start_Lat,Start_Lng,Side,City,County,State,Zipcode,Temperature(F),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,2,2019-05-21 08:29:55,34.809,-82.269,R,Greenville,Greenville,SC,29607-6027,76.000,...,False,False,False,False,False,False,Day,Day,Day,Day
1,2,2019-10-07 17:43:09,35.090,-80.746,R,Charlotte,Mecklenburg,NC,28270-8560,76.000,...,False,False,False,False,False,False,Day,Day,Day,Day
2,2,2020-12-13 21:53:00,37.146,-121.985,R,Los Gatos,Santa Clara,CA,95033,51.000,...,False,False,False,False,False,False,Night,Night,Night,Night
3,2,2018-04-17 16:51:23,39.110,-119.774,R,Carson City,Douglas,NV,89705,53.600,...,False,False,False,False,True,False,Day,Day,Day,Day
4,3,2016-08-31 17:40:49,26.103,-80.265,R,Fort Lauderdale,Broward,FL,33324,84.200,...,False,False,False,False,True,False,Day,Day,Day,Day


In [8]:
def make_df_by_year(year):
    
    registrations = pd.read_excel(f'{year}_vehicle_reg.xlsx', header = 7)
    registrations.columns = [str(i) for i in range(len(registrations.columns))]
    registrations = registrations.set_index('0')

    col1 = ['Automobiles','Buses','Trucks','Motorcycles','All Vehicles']
    col2 = ['PRIVATE/COMMERCIAL','PUBLICALLY_OWNED','TOTAL']
    col_tier = []
    for i in col1:
        for j in col2:
            col_tier.append((i,j))
    registrations.columns = pd.MultiIndex.from_tuples(col_tier)
    starting_point = registrations.index.to_list().index('Alabama')
    registrations = registrations[starting_point:].dropna()

    for i in registrations.columns:
        registrations[i] = pd.to_numeric(registrations[i])
    
    
    year_start = f"{year}-01-01"
    year_end = f"{year+1}-01-01"
    accidents_output = accidents[(accidents['Start_Time'] > year_start) & (accidents['Start_Time'] < year_end)]
    
    transform = accidents_output.groupby(['State','Severity']).count()['Start_Time'].reset_index().rename(columns = {'Start_Time':'Count'})

    count_crashes = accidents_output[['State','Severity']].groupby(['State']).count()
    count_crashes = count_crashes.reset_index()
    count_crashes['State'] = count_crashes['State'].apply(lambda x: new_states[x])
    
    #-- Make Severity
    severity_1 = []
    severity_2 = []
    severity_3 = []
    severity_4 = []

    states = list(transform['State'].unique())
    for i in states:
        state_only = transform[transform['State'] == i]
        sev_1 = state_only[state_only['Severity'] == 1]
        sev_2 = state_only[state_only['Severity'] == 2]
        sev_3 = state_only[state_only['Severity'] == 3]
        sev_4 = state_only[state_only['Severity'] == 4]

        if len(sev_1.index) > 0:
            severity_1.append(int(sev_1['Count']))
        else:
            severity_1.append(0)

        if len(sev_2.index) > 0:
            severity_2.append(int(sev_2['Count']))
        else:
            severity_2.append(0)

        if len(sev_3.index) > 0:
            severity_3.append(int(sev_3['Count']))
        else:
            severity_3.append(0)

        if len(sev_4.index) > 0:
            severity_4.append(int(sev_4['Count']))
        else:
            severity_4.append(0)
            
    state_by_severity = pd.DataFrame({'State': list(transform['State'].unique()),
             'Severity_1': severity_1,
             'Severity_2': severity_2,
             'Severity_3': severity_3,
             'Severity_4': severity_4})
    
    state_by_severity['State'] = state_by_severity['State'].apply(lambda x: new_states[x])
    count_crashes.columns = ['State','Crash Count']
    
    check = registrations['All Vehicles']['TOTAL'].to_dict()
    new_keys = []
    for k,v in check.items():
        if k.endswith(' '):
            new_keys.append(k[:-1])
        elif k.endswith(' (2)'):
            new_keys.append(k.replace(' (2)',''))
        else:
            new_keys.append(k)

    new_out = {}
    for i,j in enumerate(check.values()):
        new_out[new_keys[i]] = j
    new_out['District of Columbia'] = new_out.pop('Dist. of Col.')

    new_out.pop('Total')
    
    new_out = {
        'State': list(new_out.keys()),
        'Registered_Vehicles': list(new_out.values())
    }
    
    vehicle_reg = pd.DataFrame(new_out)
    crash_reg = count_crashes.merge(vehicle_reg, how = 'left', on = 'State').merge(state_by_severity, how = 'left', on ='State')
    crash_reg['Crash_Index'] = crash_reg['Crash Count']/ crash_reg['Registered_Vehicles'] * 10000
    return crash_reg

In [9]:
seventeen = make_df_by_year(2017)
eighteen = make_df_by_year(2018)
nineteen = make_df_by_year(2019)

In [10]:
seventeen.head()

,State,Crash Count,Registered_Vehicles,Severity_1,Severity_2,Severity_3,Severity_4,Crash_Index
0,Alabama,1813,5056073.000,3,837,842,131,3.586
1,Arkansas,95,2833697.000,0,29,13,53,0.335
2,Arizona,7976,5964434.074,4,5706,1417,849,13.373
3,California,95651,30795141.000,39,62232,31791,1589,31.060
4,Colorado,4702,5259960.000,1,2848,1162,691,8.939


In [11]:
eighteen.head()

,State,Crash Count,Registered_Vehicles,Severity_1,Severity_2,Severity_3,Severity_4,Crash_Index
0,Alabama,7544,5300199.154,7,4529,2857,151,14.233
1,Arkansas,491,2817145.296,0,201,98,192,1.743
2,Arizona,13138,5806312.675,2,9479,2803,854,22.627
3,California,98862,31022327.877,23,63912,33351,1576,31.868
4,Colorado,8981,5356017.862,1,4842,3344,794,16.768


In [12]:
nineteen.head()

,State,Crash Count,Registered_Vehicles,Severity_1,Severity_2,Severity_3,Severity_4,Crash_Index
0,Alabama,10148,5288208.452,2,7525,2471,150,19.190
1,Arkansas,925,2902110.817,0,516,227,182,3.187
2,Arizona,13633,5982559.286,3,10400,2379,851,22.788
3,California,168526,31247270.046,9,140611,26024,1882,53.933
4,Colorado,11920,5412403.746,1,4254,6455,1210,22.023


In [13]:
pd.DataFrame({'State':seventeen['State'].to_list(),
            '2017':seventeen['Crash_Index'].to_list(),
            '2018':eighteen['Crash_Index'].to_list(),
            '2019':nineteen['Crash_Index'].to_list()})

,State,2017,2018,2019
0,Alabama,3.586,14.233,19.190
1,Arkansas,0.335,1.743,3.187
2,Arizona,13.373,22.627,22.788
3,California,31.060,31.868,53.933
4,Colorado,8.939,16.768,22.023
5,Connecticut,16.201,14.855,12.465
6,District of Columbia,15.491,20.913,26.822
7,Delaware,6.593,10.154,8.929
8,Florida,22.864,22.873,19.854
9,Georgia,16.151,18.354,16.940


In [14]:
accidents.head()

,Severity,Start_Time,Start_Lat,Start_Lng,Side,City,County,State,Zipcode,Temperature(F),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,2,2019-05-21 08:29:55,34.809,-82.269,R,Greenville,Greenville,SC,29607-6027,76.000,...,False,False,False,False,False,False,Day,Day,Day,Day
1,2,2019-10-07 17:43:09,35.090,-80.746,R,Charlotte,Mecklenburg,NC,28270-8560,76.000,...,False,False,False,False,False,False,Day,Day,Day,Day
2,2,2020-12-13 21:53:00,37.146,-121.985,R,Los Gatos,Santa Clara,CA,95033,51.000,...,False,False,False,False,False,False,Night,Night,Night,Night
3,2,2018-04-17 16:51:23,39.110,-119.774,R,Carson City,Douglas,NV,89705,53.600,...,False,False,False,False,True,False,Day,Day,Day,Day
4,3,2016-08-31 17:40:49,26.103,-80.265,R,Fort Lauderdale,Broward,FL,33324,84.200,...,False,False,False,False,True,False,Day,Day,Day,Day


In [16]:
accidents.dtypes

Severity                          int64
Start_Time               datetime64[ns]
Start_Lat                       float64
Start_Lng                       float64
Side                             object
City                             object
County                           object
State                            object
Zipcode                          object
Temperature(F)                  float64
Wind_Chill(F)                   float64
Humidity(%)                     float64
Pressure(in)                    float64
Visibility(mi)                  float64
Wind_Direction                   object
Wind_Speed(mph)                 float64
Precipitation(in)               float64
Weather_Condition                object
Amenity                            bool
Bump                               bool
Crossing                           bool
Give_Way                           bool
Junction                           bool
No_Exit                            bool
Railway                            bool


In [17]:
accidents['Wind_Direction'].value_counts()

CALM        296403
Calm        225941
WNW         142701
SSW         140728
NW          138418
SW          132955
WSW         131533
SSE         130509
NNW         124828
S           110587
South       109651
SE          107141
W           105354
West        103729
ENE          97733
NE           97239
ESE          96459
NNE          95729
North        95281
N            84139
E            77838
Variable     68883
VAR          64883
East         64474
Name: Wind_Direction, dtype: int64

In [14]:
### CREATE A GRADIENT BOOSTED RANDOM FOREST MODEL 


In [ ]:
#clean data to remove unusable data
GBRF_data = accidents.drop('Start_Lat', "Start_Lng', 

In [18]:
#Assign states numbers for encoding
state_num = {
    'AL': 1,
    'AK': 2,
    'AS': 3,
    'AZ': 4,
    'AR': 5,
    'CA': 6,
    'CO': 7,
    'CT': 8,
    'DE': 9,
    'DC': 10,
    'FL': 11,
    'GA': 12,
    'GU': 13,
    'HI': 14,
    'ID': 15,
    'IL': 16,
    'IN': 17,
    'IA': 18,
    'KS': 19,
    'KY': 20,
    'LA': 21,
    'ME': 22,
    'MD': 23,
    'MA': 24,
    'MI': 25,
    'MN': 26,
    'MS': 27,
    'MO': 28,
    'MT': 29,
    'NE': 30,
    'NV': 31,
    'NH': 32,
    'NJ': 33,
    'NM': 34,
    'NY': 35,
    'NC': 36,
    'ND': 37,
    'MP': 38,
    'OH': 39,
    'OK': 40,
    'OR': 41,
    'PA': 42,
    'PR': 43,
    'RI': 44,
    'SC': 45,
    'SD': 46,
    'TN': 47,
    'TX': 48,
    'UT': 49,
    'VT': 50,
    'VI': 51,
    'VA': 52,
    'WA': 53,
    'WV': 54,
    'WI': 55,
    'WY': 56
}

In [ ]:
loans_df["month_num"] = loans_df["month"].apply(lambda x: months_num[x])
loans_df.head()

In [16]:
#Define X and y data:
X = accidents.copy()
X = X.drop('Severity', axis=1)
y = accidents['Severity']

In [ ]:
#Train/Test Split and 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()